# PITIÉ-SALPÊTRIÈRE - EDA Logistique (Interactif)

Ce notebook présente une analyse exploratoire des ressources logistiques (lits, personnel, équipements, stocks) en utilisant **Plotly** pour des visualisations interactives.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. Chargement des données
df_lits = pd.read_csv('../data/raw/lits_poles.csv')
df_perso = pd.read_csv('../data/raw/personnel_poles.csv')
df_equip = pd.read_csv('../data/raw/equipements_poles.csv')  
df_stocks = pd.read_csv('../data/raw/stocks_medicaments.csv')

# Conversion des dates
for df in [df_lits, df_perso, df_equip, df_stocks]:
    df['date'] = pd.to_datetime(df['date'])

# Calcul du taux d'occupation
df_lits['taux_occupation'] = df_lits['lits_occupes'] / df_lits['lits_totaux']

# Statistiques rapides
print("SYNTHÈSE LOGISTIQUE")
print(f"Occupation moyenne : {df_lits['taux_occupation'].mean():.1%}")
print(f"Alertes stocks      : {df_stocks['alerte_rupture'].sum():,}")

# 2. Visualisation des Pôles Critiques
perf_poles = df_lits.groupby('service')['taux_occupation'].agg(['mean','max']).reset_index()
perf_poles = perf_poles.sort_values('max', ascending=False).head(5)

fig_poles = go.Figure()
fig_poles.add_trace(go.Bar(x=perf_poles['service'], y=perf_poles['max'], name='PIC (Max)', marker_color='crimson'))
fig_poles.add_trace(go.Bar(x=perf_poles['service'], y=perf_poles['mean'], name='Moyenne', marker_color='orange'))
fig_poles.update_layout(barmode='group', title='Top 5 Pôles en Tension (Occupation)', template='plotly_dark')
fig_poles.show()

# 3. Analyse du Personnel et Absentéisme
perso_stats = df_perso.groupby('categorie')[['effectif_total', 'taux_absence']].mean().reset_index()

fig_perso = px.scatter(perso_stats, x='effectif_total', y='taux_absence', text='categorie', 
                 size='effectif_total', color='taux_absence', 
                 title='Absentéisme par Catégorie de Personnel', template='plotly_dark')
fig_perso.update_traces(textposition='top center')
fig_perso.show()

# 4. État des Stocks sur le Temps
stocks_trend = df_stocks.groupby('date')['alerte_rupture'].sum().reset_index()
fig_stocks = px.line(stocks_trend, x='date', y='alerte_rupture', title='Évolution des Alertes Stocks', 
                line_shape='spline', template='plotly_dark', color_discrete_sequence=['#00CC96'])
fig_stocks.show()

# 5. Répartition Globale des Ressources (Personnel)
fig_pie = px.pie(df_perso.groupby('categorie')['effectif_total'].sum().reset_index(), 
             values='effectif_total', names='categorie', title='Répartition Totale du Personnel', 
             hole=0.4, template='plotly_dark')
fig_pie.show()